In [1]:
import os

from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings

In [2]:
# Option 2: use an Azure OpenAI account with a deployment of an embedding model
azure_endpoint: str = "https://oai-mr-sandbox.openai.azure.com/"
azure_openai_api_key: str = "0137d1666b5945109ed858094974843b"
azure_openai_api_version: str = "2023-05-15"
azure_deployment: str = "text-embedding-ada-002" # Not sure what this does

In [3]:
vector_store_address: str = "https://ais-mr-sandbox.search.windows.net"
vector_store_password: str = "7D3JZi8Oedh5sGewI3cBFhfbJ6rEl5KKVOBEkCFv2BAzSeCDQlqj"

In [4]:
from azure.search.documents.indexes.models import (
    ScoringProfile,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    TextWeights,
)

#  Replace OpenAIEmbeddings with AzureOpenAIEmbeddings if Azure OpenAI is your provider.
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment=azure_deployment,
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_endpoint,
    api_key=azure_openai_api_key,
)
embedding_function = embeddings.embed_query

fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        filterable=True,
    ),
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=len(embedding_function("Text")),
        vector_search_profile_name="myHnswProfile",
    ),
    SearchableField(
        name="metadata",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    # Additional field to store the title
    SearchField(
        name="title",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    # Additional field to store the text
    SearchField(
        name="url",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    # Additional field for filtering on document source
    SimpleField(
        name="source_site",
        type=SearchFieldDataType.String,
        filterable=True,
    ),
    # Additional field for filtering on date
    SimpleField(
        name="date",
        type=SearchFieldDataType.DateTimeOffset,
        filterable=True,
    ),
    # Additional field for storing on keywords
    SearchField(
        name="keywords",
        type=SearchFieldDataType.Collection(SearchFieldDataType.String),
        searchable=True,
    ),
    # Additional field for filtering on premium content
    SimpleField(
        name="is_premium",
        type=SearchFieldDataType.Boolean,
        filterable=True,
    ),
]
# "langchain-vector-energy-news-v1" - text not splitted
# "langchain-vector-energy-news-v2" - text splitted
index_name: str = "langchain-vector-energy-news-v2"

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embedding_function,
    fields=fields,
)

In [5]:
vector_store.similarity_search_with_relevance_scores(query="wzrost wartości", k=3)

[(Document(page_content='Pięć związków zawodowych działających w Polskiej Grupie Górniczej domaga się negocjacji na temat wskaźnika wzrostu wynagrodzeń w 2024 i 2025 roku. W piśmie do zarządu górniczej spółki przywołują zapisy aneksu do umowy społecznej z 2021 roku, z którego wynika, że jeżeli zaistnieją „nadzwyczajne okoliczności”, przez które należy rozumieć podwyższoną inflację, to strony umowy podejmują negocjacje o wskaźniku przeciętnego miesięcznego wynagrodzenia w roku 2024 i 2025. Związkowcy piszą, że według ich oceny inflacja znacząco przekroczyła poziom, dla którego ustalono, że wynagrodzenia wzrosną o 3,4%, dlatego oczekują wzrostu wynagrodzeń na poziomie z 2023 roku. Dla przypomnienia, wzrost ten wyniósł 15,4%, co oczywiście może i ma luźny związek z inflacją, ale z wynikami PGG nie ma już żadnego. Przypomnijmy, że w tym roku PGG ma dostać dotację na pokrycie strat w wysokości 7 mld zł', metadata={'title': 'Będzie projekt odpolitycznienia spółek skarbu państwa?', 'date': '2

In [6]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = f"{azure_endpoint}/openai/deployments/gpt4_mr/chat/completions?api-version=2023-07-01-preview"
os.environ["AZURE_OPENAI_API_KEY"] = azure_openai_api_key

In [7]:
from langchain.llms import AzureOpenAI
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI

client = AzureChatOpenAI(model_name="gpt-4", temperature=0.7, max_tokens=800)
# chain = RetrievalQA.from_chain_type(llm=AzureOpenAI(azure_deployment=azure_deployment, openai_api_version=azure_openai_api_version, azure_endpoint=azure_endpoint, api_key=azure_openai_api_key), retriever=vector_store.as_retriever(), chain_type="stuff")
chain = RetrievalQA.from_chain_type(llm=client, retriever=vector_store.as_retriever(), chain_type="stuff")

In [8]:
query = "Streść artykuł o tytule: Resort klimatu przykręca śrubę. Mniej węgla, więcej OZE"
# chain.run(query)
# chain.invoke(query, retriever_kwargs={'title': "Resort klimatu przykręca śrubę. Mniej węgla, więcej OZE"})
chain.invoke(query)
# Jak się poda tytuł to pofiltuje po tytule, jak np datę to już nie, jest to kwestia tego, że 'date' jest jako SilmpleField
# a 'title' jest jako SearchField, obiekt daty nie może być SearchField

{'query': 'Streść artykuł o tytule: Resort klimatu przykręca śrubę. Mniej węgla, więcej OZE',
 'result': 'Artykuł mówi o działaniach Ministerstwa Klimatu i Środowiska w Polsce, które mają na celu zmniejszenie wydobycia węgla i zwiększenie produkcji energii ze źródeł odnawialnych (OZE). Wydobycie węgla w 2023 roku wyniosło 49 mln ton. Ministerstwo działa w ramach KPEiK, utworzonego na mocy regulacji z 2018 roku, które zobligowały państwa członkowskie UE do tworzenia krajowych planów w obszarze energii i klimatu. Resort Klimatu pracuje nad nowymi rozwiązaniami, które mają regulować te kwestie. Pomimo że propozycje Ministerstwa nie są idealne, są postrzegane jako punkt wyjścia do dyskusji. Ministerstwo Rozwoju zapowiedziało przyspieszenie prac nad liberalizacją przepisów dla linii bezpośrednich, co miało umożliwić dostarczanie energii wyłącznie z OZE. Pomimo tych zapowiedzi, resort klimatu zmienił zdanie co do obecności linii bezpośrednich w projekcie nowelizacji UC74.'}

In [9]:
# Tutaj próbuję filtrować przy pomocy tego search_kwargs pofiltrować po dacie ale też nie działa
r = vector_store.as_retriever()
r.search_kwargs = {'filters': "search.ismatch('2024-06-03', 'date')"}
# r.search_kwargs = {'filters': "search.ismatch('Resort klimatu przykręca śrubę. Mniej węgla, więcej OZE', 'title')"}
chain = RetrievalQA.from_chain_type(llm=client, retriever=r, chain_type="stuff")
query = "Streść artykuły"
chain.run(query)

c:\Users\szymo\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


HttpResponseError: () Invalid expression: The second argument to search.ismatch() is invalid. The field 'date' in the search field list is not searchable.
Parameter name: $filter
Code: 
Message: Invalid expression: The second argument to search.ismatch() is invalid. The field 'date' in the search field list is not searchable.
Parameter name: $filter